This notebook generates the dataset for the step selection model.

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import pickle
import os
import glob
import utm
from pathlib import Path
import math
from math import dist
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from osgeo import gdal
import n2w
gdal.UseExceptions()

import stepselector.preprocessing as pp

In [2]:
server_mount = '/home/blair/server/herd_hover'

In [3]:
# If obs_to_process = None, all of the raw track files in the data/raw_tracks folder will be processed
obs_to_process = None

# Alternatively, a subset of observations can be processed
#obs_to_process = ['ob015', 'ob027', 'ob053', 'ob074', 'ob088', 'ob090']

# What step length to use?
step_length = 10
dataset = n2w.convert(step_length) + '_meter_steps'

In [4]:
# Define directories
data_folder = os.path.join(server_mount, 'zebra_movement_data')
raw_tracks_directory = os.path.join(data_folder, 'raw_tracks')
rasters_directory = os.path.join(data_folder, 'rasters')
observed_steps_directory = os.path.join(data_folder, dataset, 'observed')
simulated_steps_directory = os.path.join(data_folder, dataset, 'simulated')
viewshed_save_directory = os.path.join(data_folder, 'visual_fields', dataset)
map_directory = os.path.join(server_mount, '3D_mapping/big_maps')

# Define files
ob_metadata_file = os.path.join(data_folder, 'observation_metadata.csv')
track_metadata_file = os.path.join(data_folder, 'track_metadata.csv')

In [25]:
# Step 1 - extract observed steps
pp.extract_observed_steps(step_length = step_length, 
                       raw_tracks_directory = raw_tracks_directory, 
                       save_directory = observed_steps_directory,
                       rasters_directory = rasters_directory,
                       #ob_metadata_file = ob_metadata_file, # change to ob_metadata_file if there's risk of observed steps falling outside mapped area
                       obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
# Step 2 - simulate fake steps
pp.simulate_fake_steps(n_steps = 20, 
                    observed_steps_directory = observed_steps_directory, 
                    save_directory = simulated_steps_directory, 
                    rasters_directory = rasters_directory, 
                    ob_metadata_file = ob_metadata_file, 
                    obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# Step 3 - get observer and step info
pp.get_observer_and_step_info(observed_steps_directory = observed_steps_directory,
                           simulated_steps_directory = simulated_steps_directory,
                           ob_metadata_file = ob_metadata_file,
                           obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
# Step 4 - calculate zebra observation heights for visibility analyses
pp.calculate_zebra_heights(observed_steps_directory = observed_steps_directory,
                        simulated_steps_directory = simulated_steps_directory,
                        rasters_directory = rasters_directory,
                        ob_metadata_file = ob_metadata_file,
                        obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
# Step 5 - get road info
pp.road_or_no(observed_steps_directory = observed_steps_directory,
              simulated_steps_directory = simulated_steps_directory,
              rasters_directory = rasters_directory,
              ob_metadata_file = ob_metadata_file,
              obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
# Step 6 - get step slope
pp.step_slope(observed_steps_directory = observed_steps_directory,
           simulated_steps_directory = simulated_steps_directory,
           rasters_directory = rasters_directory,
           ob_metadata_file = ob_metadata_file,
           obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
# Step 7 - get social info
pp.get_social_info(observed_steps_directory = observed_steps_directory,
                simulated_steps_directory = simulated_steps_directory,
                raw_tracks_directory = raw_tracks_directory,
                rasters_directory = rasters_directory,
                ob_metadata_file = ob_metadata_file,
                track_metadata_file = track_metadata_file,
                social_radius = 10,
                obs_to_process = obs_to_process)

  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# Step 8 - get ground cover
pp.get_ground_cover(observed_steps_directory = observed_steps_directory,
              simulated_steps_directory = simulated_steps_directory,
              rasters_directory = rasters_directory,
              obs_to_process = obs_to_process)

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
# Step 9 - get track info
pp.get_track_info(observed_steps_directory = observed_steps_directory,
                  simulated_steps_directory = simulated_steps_directory,
                  track_metadata_file = track_metadata_file,
                  obs_to_process = obs_to_process)

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# Step 10 - generate viewsheds
pp.preprocess_viewsheds(observed_steps_directory = observed_steps_directory,
                  simulated_steps_directory = simulated_steps_directory,
                  ob_metadata_file = ob_metadata_file,
                  obs_to_process = obs_to_process,
                  map_directory = map_directory,
                  viewshed_save_directory = viewshed_save_directory,
                  keep_rasters = True,
                  radius = 30, # viewshed radius in meters
                  threads = 16) # how many threads to use to generate viewsheds

starting /home/blair/server/herd_hover/zebra_movement_data/ten_meter_steps/observed/ob015_track00_10msteps.pkl:  2024-12-05 14:03:36.403235


KeyboardInterrupt: 